# Using Vertex AI Vizier to optimize multiple objectives


## Introduction

This example demonstrates [Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview) multi-objective optimization. Multi-objective optimization is concerned with mathematical optimization problems involving more than one objective function to be optimized simultaneously. 
Vertex AI Vizier is a black-box optimization service. You will often see Vertex AI Vizier used to optimize hyperparameters of ML models, but it can also perform other optimization tasks— as illustrated by this example.

You can use the default Terra Cloud Environment for this example.

### 'Native' GCP project required

This example requires a ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra).  The Vizier service will be run using that project, not your Terra workspace project.

### Optimization objective

The goal is to __`minimize`__ the objective metric:
   ```
   y1 = r*sin(theta)
   ```

and simultaneously __`maximize`__ the objective metric:
   ```
   y2 = r*cos(theta)
   ```

that you will evaluate over the parameter space:

   - __`r`__ in [0,1],

   - __`theta`__ in [0, pi/2]
   
### Overview of running a Vizier *study*

Here is an overview of the process for setting up and running a Vizier study. The specifics are in the code below.

- Create a *study* configuration. This includes info on the parameter(s) that you want to tune,your objective metric(s), and the HP search algorithm to use.
- Create a Vizier client object via the `aiplatform` library, which you'll use to interact with the service.
- Create a Vizier *study* via the client object
- Define function(s) to evaluate the objective metric(s).
- Run the trials by interacting with the Vizier client to get "suggested" trial param sets. Run your evaluation functions using the suggested trial param sets.
- Record the outcome of a given trial with the Vizier client. You use a `client_id` to indicate the identifier of the client that is requesting the suggestion. If multiple suggestion requests have the same `client_id`, the service will return the identical suggested trial if the trial is `PENDING`, and provide a new trial if the last suggested trial was completed.
- Request information from Vizier about the optimal trials. It returns the pareto-optimal trials for a multi-objective study, or the optimal trials for a single-objective study.   

### Costs

This tutorial uses Vertex AI Vizier. Pricing information is [here](https://cloud.google.com/vertex-ai/pricing#vizier).
For this simple example, which uses `RANDOM_SEARCH`, there should not be a charge to your 'native' GCP project.


## Setup

### Install the Vertex AI library

Download and install the Vertex AI library. (On the default Terra image, this should already be installed, though this command will update it if need be).

In [ ]:
! unset PIP_TARGET ; pip install --user --upgrade google-cloud-aiplatform

In [ ]:
# Restart the kernel after pip installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Set up your 'native' Google Cloud project

1. If you have not already done so, follow the instructions [here](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra) for a GCP project, including creating a Terra group as necessary, and then adding your Terra group on the Google project. You'll need to enable billing as described in the article.

2. Then, [enable the Vertex AI APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) for your project.

You'll need to fill in the project ID for your native project in the next cell.


### Import libraries and define constants

**Before you run the following cell**, set the `PROJECT_ID` for your ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra).

In [ ]:
REGION = "us-central1"
# Set your 'native' GCP project ID here, NOT your workspace project ID
PROJECT_ID = "YOUR-NATIVE-PROJECT-ID"  # CHANGE THIS

In [ ]:
import datetime
import json

from google.cloud import aiplatform

## Define and run a Vizier *study*


Set some variables.  Ensure that your `PROJECT_ID` is set correctly.

In [ ]:
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)  # @param {type: 'string'}
ENDPOINT = REGION + "-aiplatform.googleapis.com"
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("ENDPOINT: {}".format(ENDPOINT))
print("REGION: {}".format(REGION))
print("PARENT: {}".format(PARENT))

### Create the *study* configuration

The study configuration is built as a hierarchical python dictionary. It is already filled out. Run the cell to configure the study.

In [ ]:
# Parameter Configuration

param_r = {"parameter_id": "r", "double_value_spec": {"min_value": 0, "max_value": 1}}

param_theta = {
    "parameter_id": "theta",
    "double_value_spec": {"min_value": 0, "max_value": 1.57},
}

# Objective Metrics
metric_y1 = {"metric_id": "y1", "goal": "MINIMIZE"}

# Objective Metrics
metric_y2 = {"metric_id": "y2", "goal": "MAXIMIZE"}

# Put it all together in a study configuration
study = {
    "display_name": STUDY_DISPLAY_NAME,
    "study_spec": {
        "algorithm": "RANDOM_SEARCH",
        "parameters": [
            param_r,
            param_theta,
        ],
        "metrics": [metric_y1, metric_y2],
    },
}

print(json.dumps(study, indent=2, sort_keys=True))

### Create the Vizier *study*

Next, create the study.  You'll run it below, to optimize the two objectives.

In [ ]:
vizier_client = aiplatform.gapic.VizierServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)
study = vizier_client.create_study(parent=PARENT, study=study)
STUDY_ID = study.name
print("STUDY_ID: {}".format(STUDY_ID))

### Metric evaluation functions

Next, define some functions to evaluate the two objective metrics.

In [ ]:
import math


# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(trial_id, r, theta):
    print(("=========== Start Trial: [{}] =============").format(trial_id))

    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    metric1 = {"metric_id": "y1", "value": y1}
    metric2 = {"metric_id": "y2", "value": y2}

    # Return the results for this trial
    return [metric1, metric2]

### Set configuration parameters for running trials

__`client_id`__: The identifier of the client that is requesting the suggestion. If multiple suggestion requests have the same `client_id`, the service will return the identical suggested trial if the trial is `PENDING`, and provide a new trial if the last suggested trial was completed.

__`suggestion_count_per_request`__: The number of suggestions (trials) requested in a single request.

__`max_trial_id_to_stop`__: The number of trials to explore before stopping. It is set to 15 to shorten the time to run the example, so it is not expected to converge. For convergence, it would likely need to be about 20 (a good rule of thumb is to multiply the total dimensionality by 10).


In [ ]:
client_id = "client1"
suggestion_count_per_request = 5
max_trial_id_to_stop = 15

print("client_id: {}".format(client_id))
print("suggestion_count_per_request: {}".format(suggestion_count_per_request))
print("max_trial_id_to_stop: {}".format(max_trial_id_to_stop))

### Run Vertex Vizier trials

Run the trials.

In [ ]:
trial_id = 0
while int(trial_id) < max_trial_id_to_stop:
    suggest_response = vizier_client.suggest_trials(
        {
            "parent": STUDY_ID,
            "suggestion_count": suggestion_count_per_request,
            "client_id": client_id,
        }
    )

    for suggested_trial in suggest_response.result().trials:
        trial_id = suggested_trial.name.split("/")[-1]
        trial = vizier_client.get_trial({"name": suggested_trial.name})

        if trial.state in ["COMPLETED", "INFEASIBLE"]:
            continue

        for param in trial.parameters:
            if param.parameter_id == "r":
                r = param.value
            elif param.parameter_id == "theta":
                theta = param.value
        print("Trial : r is {}, theta is {}.".format(r, theta))

        vizier_client.add_trial_measurement(
            {
                "trial_name": suggested_trial.name,
                "measurement": {
                    "metrics": CreateMetrics(suggested_trial.name, r, theta)
                },
            }
        )

        response = vizier_client.complete_trial(
            {"name": suggested_trial.name, "trial_infeasible": False}
        )

### List the optimal solutions

`list_optimal_trials` returns the [pareto-optimal](https://en.wikipedia.org/wiki/Pareto_efficiency) trials for a multi-objective study, or the optimal trials for single-objective study. For this example, since we defined a multi-objective study, pareto-optimal trials will be returned.

In [ ]:
optimal_trials = vizier_client.list_optimal_trials({"parent": STUDY_ID})

print("optimal_trials: {}".format(optimal_trials))

### View your trial results in the Vertex AI UI

To view and compare your trial results, you can visit the [Vertex AI "Experiments" panel](https://console.cloud.google.com/vertex-ai/experiments/experiments) in the GCP Cloud Console, then click on the **VIZIER STUDIES** tab. From there, click in to a "Study name".

You can view info about the trial metrics as they are run.

## Listing, fetching, and deleting studies



In [ ]:
# To list studies in a specific project and region, send the following request:

vizier_client.list_studies({"parent": PARENT})

In [ ]:
# To get a study, send the following request:

vizier_client.get_study({"name": STUDY_NAME})

In [ ]:
# To delete a study:
vizier_client.delete_study({"name": STUDY_ID})

---
Copyright 2022 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style    
license that can be found in the LICENSE file or at    
https://developers.google.com/open-source/licenses/bsd